In [1]:
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import scipy.stats as stats 
from scipy.stats import shapiro
import warnings
from scipy.stats import mannwhitneyu
import numpy as np
warnings.filterwarnings('ignore')

In [2]:
air_bnb_path = "../Data-Project-1/Ashville listings.csv"
air_bnb_asheville = pd.read_csv(air_bnb_path, encoding='latin-1')
air_bnb_asheville.head()
air_bnb_asheville = air_bnb_asheville[air_bnb_asheville["number_of_reviews"] > 5]
air_bnb_asheville

,id,listing_url,name,bedrooms,beds,bathrooms,accommodates,price,room_type,review_scores_rating,...,host_since,current_host_listings,total_host_listings_count,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
1,6.190000e+17,https://www.airbnb.com/rooms/619419707858005894,Home in Asheville,1 bedroom,1 beds,1 bath,2,137,Entire home/apt,5.00,...,3/20/2017,1528,2351,2,12,8/19/2023,0.84,3,2,8
2,6.470000e+17,https://www.airbnb.com/rooms/647499312875397788,Rental unit in Asheville,1 bedroom,1 beds,1 bath,2,91,Entire home/apt,5.00,...,3/20/2017,1528,2351,2,6,10/28/2022,0.52,1,0,6
3,7.870000e+17,https://www.airbnb.com/rooms/786892739148623801,Rental unit in Asheville,1 bedroom,2 beds,1 bath,4,116,Entire home/apt,5.00,...,3/20/2017,1528,2351,1,20,9/10/2023,6.82,1,26,20
4,8.090000e+17,https://www.airbnb.com/rooms/808832407323703451,Home in Arden,1 bedroom,1 beds,1 bath,2,75,Entire home/apt,5.00,...,3/20/2017,1528,2351,2,7,9/4/2023,3.96,1,286,7
17,3.893117e+07,https://www.airbnb.com/rooms/38931169,Condo in Asheville,1 bedroom,2 beds,1 bath,4,204,Entire home/apt,4.93,...,9/20/2017,509,1165,3,28,8/20/2023,1.86,1,280,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3280,9.200000e+17,https://www.airbnb.com/rooms/920404826869152671,Home in Asheville,NaN,10 beds,6 baths,16,302,Entire home/apt,3.97,...,9/19/2022,48,56,2,33,6/19/2023,1.22,17,3,11
3281,4.841331e+07,https://www.airbnb.com/rooms/48413314,Bed and breakfast in Asheville,NaN,10 beds,6 baths,16,1650,Private room,3.33,...,10/2/2012,7,11,1,6,9/5/2022,0.21,51,334,0
3282,3.055866e+07,https://www.airbnb.com/rooms/30558665,Home in Asheville,NaN,1 beds,1 bath,2,106,Private room,4.92,...,5/3/2012,3,5,2,367,9/4/2023,4.71,4,49,40
3283,6.170000e+17,https://www.airbnb.com/rooms/617373580483490671,Chalet in Asheville,NaN,NaN,2 baths,9,361,Entire home/apt,4.80,...,3/1/2022,3,3,2,10,5/20/2023,0.36,1,83,4


In [3]:
# Group DataFrame by Host ID

host_ratings = air_bnb_asheville.groupby(['host_id', 'host_name', 'current_host_listings', 'total_host_listings_count'])['review_scores_rating'].mean()
host_ratings_df = pd.DataFrame(host_ratings)
host_ratings_dfs = host_ratings_df.sort_values(by='total_host_listings_count', ascending=False)
host_ratings_dfs.reset_index(inplace=True)
host_ratings_dfs = host_ratings_dfs.dropna(subset=['review_scores_rating'])
print("mean scores rating: " + str(host_ratings_dfs["review_scores_rating"].mean()))
host_ratings_dfs

mean scores rating: 4.84398339106618


,host_id,host_name,current_host_listings,total_host_listings_count,review_scores_rating
0,121681645,Evolve,1528,2351,4.787576
1,124060715,Evolve,673,1485,4.840000
2,456511812,Evolve,1173,1444,4.700000
3,151397468,Evolve,509,1165,4.730000
4,206690366,Vacasa North Carolina,624,1118,4.730000
...,...,...,...,...,...
1777,23952961,Caroline,1,1,4.940000
1778,129158183,Meredith,1,1,4.580000
1779,23909518,Rebecca,1,1,4.890000
1780,130138003,Kristyn,1,1,4.830000


In [4]:
# Calculated Average Score Ratings for Each Host(Host ID) with only One Listing

single_listing_host_ratings_dfs = host_ratings_dfs.loc[host_ratings_dfs["total_host_listings_count"] == 1]

# Drop rows with NaN values
single_listing_host_ratings_dfs = single_listing_host_ratings_dfs.dropna()

#Caclulated mean

print("mean scores rating: " + str(single_listing_host_ratings_dfs["review_scores_rating"].mean()))
single_listing_host_ratings_dfs.head()

mean scores rating: 4.858742791234142


,host_id,host_name,current_host_listings,total_host_listings_count,review_scores_rating
915,7523601,Karen,1,1,4.85
916,12642086,Briana,1,1,4.91
917,266624774,Na,1,1,4.66
918,7502088,Karen,1,1,4.66
919,187473767,Drew And Caroline,1,1,4.90


In [5]:
small_host_ratings_dfs = host_ratings_dfs[
    (host_ratings_dfs["total_host_listings_count"] > 2) & 
    (host_ratings_dfs["total_host_listings_count"] < 10)]

# Drop rows with NaN values
small_host_ratings_dfs = small_host_ratings_dfs.dropna()

# Calculate mean scores rating
mean_score_rating = small_host_ratings_dfs["review_scores_rating"].mean()
print("Mean Scores Rating: " + str(mean_score_rating))

# Display the first few rows of the DataFrame
small_host_ratings_dfs.head()

Mean Scores Rating: 4.827862638026224


,host_id,host_name,current_host_listings,total_host_listings_count,review_scores_rating
89,32195328,Harshad And Ishita,8,9,4.510000
90,5451404,Amy,8,9,4.970000
91,7625694,John,9,9,4.727778
92,4635300,Lauren,4,9,4.820000
93,8594089,Kat,9,9,4.840000


In [6]:
# Calculated Average Score Ratings for Each Host(Host ID) with Total Listings Greater than 10

large_host_ratings = host_ratings_dfs.loc[host_ratings_dfs["total_host_listings_count"] > 10]

# Drop rows with NaN values
large_host_ratings_dfs = large_host_ratings.dropna()

#Caclulated mean

print("mean scores rating: " + str(large_host_ratings_dfs["review_scores_rating"].mean()))
large_host_ratings_dfs.head()

mean scores rating: 4.775283490226457


,host_id,host_name,current_host_listings,total_host_listings_count,review_scores_rating
0,121681645,Evolve,1528,2351,4.787576
1,124060715,Evolve,673,1485,4.840000
2,456511812,Evolve,1173,1444,4.700000
3,151397468,Evolve,509,1165,4.730000
4,206690366,Vacasa North Carolina,624,1118,4.730000


In [7]:
# Gather Each group of Hosts Data for Significance Test

total_ratings_df = host_ratings_dfs["review_scores_rating"]
single_host_df = single_listing_host_ratings_dfs["review_scores_rating"]
small_host_df = small_host_ratings_dfs["review_scores_rating"]
large_host_df = large_host_ratings_dfs["review_scores_rating"]

In [8]:
print("Single Group: " + str(stats.ttest_1samp(single_host_df, total_ratings_df.mean())))
print("Small Group: " + str(stats.ttest_1samp(small_host_df, total_ratings_df.mean())))
print("Large Group: " + str(stats.ttest_1samp(large_host_df, total_ratings_df.mean())))

Single Group: TtestResult(statistic=3.5565510386260626, pvalue=0.0003961368339184739, df=866)
Small Group: TtestResult(statistic=-2.6375368846726936, pvalue=0.00863452103467443, df=459)
Large Group: TtestResult(statistic=-3.9596757059386216, pvalue=0.00017262613116845278, df=73)


In [9]:
#ANOVA Test

stats.f_oneway(total_ratings_df, single_host_df, small_host_df, large_host_df) 

F_onewayResult(statistic=13.753739862695113, pvalue=6.53223390306174e-09)

In [10]:
# Conduct the Kruskal-Wallis Test 

result = stats.kruskal(total_ratings_df, single_host_df, small_host_df, large_host_df)
result

KruskalResult(statistic=48.22842472120341, pvalue=1.903980455848072e-10)

In [11]:
total_df_ratings = host_ratings_dfs["review_scores_rating"].mean()
single_host = single_listing_host_ratings_dfs["review_scores_rating"].mean()
small_host = small_host_ratings_dfs["review_scores_rating"].mean()
large_host = large_host_ratings_dfs["review_scores_rating"].mean()

In [12]:
data = [
    ['Single_Host', single_host, total_df_ratings],
    ['Small_Host(between 2-10 Listings)', small_host, total_df_ratings],
    ['Large_Host(over 10 Listings)', large_host, total_df_ratings],
    
]

columns = ['Type of Host', 'Review Scores Rating', 'Average Score of Total']

df_a = pd.DataFrame(data, columns=columns)
df_a['Review Scores Rating'] = df_a['Review Scores Rating'].round(4)
df_a['Average Score of Total'] = df_a['Average Score of Total'].round(4)
df_a

,Type of Host,Review Scores Rating,Average Score of Total
0,Single_Host,4.8587,4.844
1,Small_Host(between 2-10 Listings),4.8279,4.844
2,Large_Host(over 10 Listings),4.7753,4.844


In [13]:
air_bnb_path = "../Data-Project-1/Columbus_listings.csv"
air_bnb_columbus = pd.read_csv(air_bnb_path, encoding='latin-1')
air_bnb_columbus = air_bnb_columbus[air_bnb_columbus["number_of_reviews"] > 5]
air_bnb_columbus.head()

,id,name,bedrooms,beds,bathrooms,host_id,host_name,host_since,host_location,host_response_time,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_total_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5.202931e+07,Guest suite in Columbus,Studio,1 bed,1 bath,3023813,Max,7/23/2012,"Columbus, OH",within an hour,...,4.90,4.73,4.52,2021-2326,f,2,2,0,0,2.10
1,7.060000e+17,Rental unit in Columbus,Studio,1 bed,1 bath,8963103,Sarah,9/21/2013,"Columbus, OH",within an hour,...,5.00,5.00,4.96,2021-2545,f,18,18,0,0,3.35
2,4.790022e+07,Home in Columbus,Studio,1 bed,1 bath,10620281,Robert,12/14/2013,"Columbus, OH",within an hour,...,4.94,4.99,4.79,2023-2084,f,6,6,0,0,3.37
3,1.426238e+07,Rental unit in Columbus,Studio,1 bed,1 bath,14759634,Leisa,4/26/2014,"Columbus, OH",within an hour,...,4.94,4.90,4.63,2019-2921,f,1,1,0,0,1.68
4,2.913068e+07,Guest suite in Columbus,Studio,1 bed,1 bath,16967531,Philip,6/18/2014,"Columbus, OH",within an hour,...,5.00,4.98,4.96,NaN,f,1,1,0,0,10.10


In [14]:
host_ratings_c = air_bnb_columbus.groupby(['host_id', 'host_name', 'current_host_listings', 'total_host_listings_count'])['review_scores_rating'].mean()
host_ratings_df_c = pd.DataFrame(host_ratings_c)
host_ratings_df_c = host_ratings_df_c.sort_values(by='total_host_listings_count', ascending=False)
host_ratings_df_c.reset_index(inplace=True)
host_ratings_df_c = host_ratings_df_c.dropna(subset=['review_scores_rating'])
print("mean scores rating: " + str(host_ratings_df_c["review_scores_rating"].mean()))
host_ratings_df_c

mean scores rating: 4.845628410032191


,host_id,host_name,current_host_listings,total_host_listings_count,review_scores_rating
0,121719854,Evolve,1567,2554,4.151667
1,447235921,Traverse,423,550,4.660000
2,158439641,GH Hospitality,106,144,4.633902
3,235497311,Marc (TailwindSTR),63,130,4.587632
4,325932974,Marc And Darius,63,130,4.498125
...,...,...,...,...,...
707,77828439,Lola,1,1,4.930000
708,78005606,Kelly,1,1,4.960000
709,78518759,Steven,1,1,4.920000
710,79158175,Joy,1,1,5.000000


In [15]:
# Calculated Average Score Ratings for Each Host(Host ID) with only One Listing

single_listing_host_ratings_dfs_c = host_ratings_df_c.loc[host_ratings_df_c["total_host_listings_count"] == 1]

# Drop rows with NaN values
single_listing_host_ratings_dfs_c = single_listing_host_ratings_dfs_c.dropna()

#Caclulated mean

print("mean scores rating: " + str(single_listing_host_ratings_dfs_c["review_scores_rating"].mean()))
single_listing_host_ratings_dfs_c.head()

mean scores rating: 4.866279069767442


,host_id,host_name,current_host_listings,total_host_listings_count,review_scores_rating
454,32912159,John,1,1,4.74
455,42426080,Seth,1,1,4.89
456,423469919,Sayam,1,1,4.25
457,387439145,Raven,1,1,4.79
458,423549973,Karl,1,1,4.70


In [16]:
# Calculated Average Score Ratings for Each Host(Host ID) with only One Listing

small_host_ratings_c = host_ratings_df_c[ (host_ratings_df_c["total_host_listings_count"] > 2) &  (host_ratings_df_c["total_host_listings_count"] < 10)]

# Drop rows with NaN values
small_host_ratings_c = small_host_ratings_c.dropna()

#Caclulated mean

print("mean scores rating: " + str(small_host_ratings_c["review_scores_rating"].mean()))
small_host_ratings_c.head()

mean scores rating: 4.834767807122848


,host_id,host_name,current_host_listings,total_host_listings_count,review_scores_rating
77,79004963,Amy And Kevin,7,9,4.940000
78,431344669,Jesse,9,9,4.940000
79,426481609,Sherry,8,9,4.795000
80,119659878,Kay,8,9,4.598571
81,403687886,Shan,7,9,4.887500


In [17]:
# Calculated Average Score Ratings for Each Host(Host ID) with Total Listings Greater than 10

large_host_ratings_c = host_ratings_df_c.loc[host_ratings_df_c["total_host_listings_count"] > 10]

# Drop rows with NaN values
large_host_ratings_c = large_host_ratings_c.dropna()

#Caclulated mean

print("mean scores rating: " + str(large_host_ratings_c["review_scores_rating"].mean()))
large_host_ratings_c.head()

mean scores rating: 4.757435118565439


,host_id,host_name,current_host_listings,total_host_listings_count,review_scores_rating
0,121719854,Evolve,1567,2554,4.151667
1,447235921,Traverse,423,550,4.660000
2,158439641,GH Hospitality,106,144,4.633902
3,235497311,Marc (TailwindSTR),63,130,4.587632
4,325932974,Marc And Darius,63,130,4.498125


In [18]:
# Gather Each group of Hosts Data for Significance Test

total_ratings_df_c = host_ratings_df_c["review_scores_rating"]
single_host_df_c = single_listing_host_ratings_dfs_c["review_scores_rating"]
small_host_df_c = small_host_ratings_c["review_scores_rating"]
large_host_df_c = large_host_ratings_c["review_scores_rating"]

In [19]:
# ANOVA Testing

stats.f_oneway(total_ratings_df_c, single_host_df_c, small_host_df_c, large_host_df_c) 

F_onewayResult(statistic=8.2411076977143, pvalue=1.9679811184596575e-05)

In [20]:
# Conduct the Kruskal-Wallis Test 

result = stats.kruskal(total_ratings_df_c, single_host_df_c, small_host_df_c, large_host_df_c)
result

KruskalResult(statistic=33.1767224958914, pvalue=2.955745973144784e-07)

In [21]:
print("Single Group: " + str(stats.ttest_1samp(single_host_df_c, total_ratings_df_c.mean())))
print("Small Group: " + str(stats.ttest_1samp(small_host_df_c, total_ratings_df_c.mean())))
print("Large Group: " + str(stats.ttest_1samp(large_host_df_c, total_ratings_df_c.mean())))

Single Group: TtestResult(statistic=2.1061521346829, pvalue=0.03616105105250365, df=257)
Small Group: TtestResult(statistic=-0.9999558535708566, pvalue=0.3183517234154841, df=237)
Large Group: TtestResult(statistic=-3.8692213823359998, pvalue=0.00024719725298886694, df=68)


In [22]:
total_df_ratings_c = host_ratings_df_c["review_scores_rating"].mean()
single_host_c = single_listing_host_ratings_dfs_c["review_scores_rating"].mean()
small_host_c = small_host_ratings_c["review_scores_rating"].mean()
large_host_c = large_host_ratings_c["review_scores_rating"].mean()

In [23]:
data_c = [
    ['Single_Host', single_host_c, total_df_ratings_c],
    ['Small_Host(between 2-10 Listings)', small_host_c, total_df_ratings_c],
    ['Large_Host(over 10 Listings)', large_host_c, total_df_ratings_c],
    
]

columns = ['Type of Host', 'Review Scores Rating', 'Average Score of Total']

df_c = pd.DataFrame(data_c, columns=columns)
df_c['Review Scores Rating'] = df_c['Review Scores Rating'].round(4)
df_c['Average Score of Total'] = df_c['Average Score of Total'].round(4)
df_c

,Type of Host,Review Scores Rating,Average Score of Total
0,Single_Host,4.8663,4.8456
1,Small_Host(between 2-10 Listings),4.8348,4.8456
2,Large_Host(over 10 Listings),4.7574,4.8456


In [24]:
air_bnb_path = "../Data-Project-1/ri.listings.csv"
air_bnb_ri = pd.read_csv(air_bnb_path, encoding='latin-1')
air_bnb_ri = air_bnb_ri[air_bnb_ri["number_of_reviews"] > 5]
air_bnb_ri.head()

,id,name,bedrooms,beds,baths,host_id,host_name,host_since,host_location,host_response_time,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
20,4.240441e+07,Rental unit in Providence,1 bedroom,1 bed,1 shared bath,201374777,Shaina,7/10/18,"Somerville, MA",within an hour,...,4.50,4.50,3.83,NaN,f,16,0,16,0,0.15
22,8.173450e+17,Rental unit in Newport,1 bedroom,1 bed,1 bath,498868630,Vitaly,1/31/23,NaN,within an hour,...,4.00,4.64,3.73,NaN,t,13,13,0,0,1.51
25,1.726898e+07,Home in Warwick,1 bedroom,1 bed,2 shared baths,61220795,Andrey,3/2/16,"North Providence, RI",within a few hours,...,4.29,4.43,3.71,NaN,f,1,0,1,0,0.10
30,4.108185e+07,Rental unit in Providence,1 bedroom,1 bed,2 shared baths,201374777,Shaina,7/10/18,"Somerville, MA",within an hour,...,4.00,4.57,4.57,NaN,f,16,0,16,0,0.26
48,5.057018e+07,Rental unit in Middletown,1 bedroom,1 bed,1 bath,407327876,Rheas,6/15/21,NaN,within an hour,...,4.32,4.42,4.16,NaN,f,8,7,1,0,0.71


In [25]:
host_ratings_ri = air_bnb_ri.groupby(['host_id', 'host_name', 'current_host_listings', 'total_host_listings_count'])['review_scores_rating'].mean()
host_ratings_df_ri = pd.DataFrame(host_ratings_ri)
host_ratings_dfs_ri = host_ratings_df_ri.sort_values(by='total_host_listings_count', ascending=False)
host_ratings_dfs_ri = host_ratings_dfs_ri.dropna(subset=['review_scores_rating'])
host_ratings_dfs_ri.reset_index(inplace=True)
host_ratings_dfs_ri.head()

,host_id,host_name,current_host_listings,total_host_listings_count,review_scores_rating
0,442029804,Live Suite,2709,2925,4.882000
1,456511812,Evolve,1169,1444,5.000000
2,389697794,David,1008,1070,4.965000
3,373042041,RedAwning,406,983,3.940000
4,122377513,Evolve,451,932,4.627429


In [26]:
single_host_ratings = host_ratings_dfs_ri.loc[host_ratings_dfs_ri["total_host_listings_count"] == 1]
small_host_ratings_ri = host_ratings_dfs_ri[ (host_ratings_dfs_ri["total_host_listings_count"] > 2) &  (host_ratings_dfs_ri["total_host_listings_count"] < 10)]
large_host_ratings_ri = host_ratings_dfs_ri.loc[host_ratings_dfs_ri["total_host_listings_count"] > 10]

# Drop NA Values
large_host_drop_ri = large_host_ratings_ri.dropna(subset=['review_scores_rating'])
small_host_drop_ri = small_host_ratings_ri.dropna(subset=['review_scores_rating'])
single_host_drop_ri = single_host_ratings.dropna(subset=['review_scores_rating'])


In [27]:
print("mean scores rating: " + str(single_host_drop_ri["review_scores_rating"].mean()))
single_host_drop_ri.head()

mean scores rating: 4.892775628626692


,host_id,host_name,current_host_listings,total_host_listings_count,review_scores_rating
1132,507129597,Marlayna,1,1,5.00
1133,508023066,Michael,1,1,4.95
1134,508427997,Jennifer,1,1,5.00
1135,246509054,Shelagh,1,1,4.95
1136,509646247,Judy,1,1,4.89


In [28]:
print("mean scores rating: " + str(small_host_drop_ri["review_scores_rating"].mean()))
small_host_drop_ri.head()

mean scores rating: 4.844142382905123


,host_id,host_name,current_host_listings,total_host_listings_count,review_scores_rating
105,223101531,Marcelo,4,9,4.640000
106,349685463,Jonathan,9,9,4.797500
107,32081408,David,7,9,4.775714
108,265021925,Poli,9,9,4.844444
109,199058580,Michele,3,9,4.640000


In [29]:
print("mean scores rating: " + str(large_host_drop_ri["review_scores_rating"].mean()))
large_host_drop_ri.head()

mean scores rating: 4.756379150635263


,host_id,host_name,current_host_listings,total_host_listings_count,review_scores_rating
0,442029804,Live Suite,2709,2925,4.882000
1,456511812,Evolve,1169,1444,5.000000
2,389697794,David,1008,1070,4.965000
3,373042041,RedAwning,406,983,3.940000
4,122377513,Evolve,451,932,4.627429


In [30]:
# Gather for Statistical Testing

total_ratings_df_ri = host_ratings_dfs_ri["review_scores_rating"]
large_host_ri = large_host_drop_ri["review_scores_rating"]
small_host_ri = small_host_drop_ri["review_scores_rating"]
single_host_ri = single_host_drop_ri["review_scores_rating"]

In [31]:
# ANOVA Testing

stats.f_oneway(total_ratings_df_ri, single_host_ri, small_host_ri, large_host_ri) 

F_onewayResult(statistic=30.874361808031015, pvalue=1.0225777677379221e-19)

In [32]:
result = stats.kruskal(total_ratings_df_ri, single_host_ri, small_host_ri, large_host_ri)
result

KruskalResult(statistic=99.71355554488531, pvalue=1.7909580318719815e-21)

In [33]:
print("Single Group: " + str(stats.ttest_1samp(single_host_ri, total_ratings_df_ri.mean())))
print("Small Group: " + str(stats.ttest_1samp(small_host_ri, total_ratings_df_ri.mean())))
print("Large Group: " + str(stats.ttest_1samp(large_host_ri, total_ratings_df_ri.mean())))

Single Group: TtestResult(statistic=4.6510961452277755, pvalue=3.729744722698754e-06, df=1033)
Small Group: TtestResult(statistic=-4.026434421065327, pvalue=6.431741556724639e-05, df=568)
Large Group: TtestResult(statistic=-5.322056099741338, pvalue=7.325283690650016e-07, df=91)


In [34]:
total_df_ratings_avg_ri = host_ratings_dfs_ri["review_scores_rating"].mean()
single_host_avg_ri = single_host_drop_ri["review_scores_rating"].mean()
small_host_avg_ri = small_host_drop_ri["review_scores_rating"].mean()
large_host_avg_ri = large_host_drop_ri["review_scores_rating"].mean()

In [35]:
data_ri = [
    ['Single_Host', single_host_avg_ri, total_df_ratings_avg_ri],
    ['Small_Host(between 2-10 Listings)', small_host_avg_ri, total_df_ratings_avg_ri],
    ['Large_Host(over 10 Listings)', large_host_avg_ri, total_df_ratings_avg_ri],
    
]

columns = ['Type of Host', 'Review Scores Rating', 'Average Score of Total']

df_ri = pd.DataFrame(data_ri, columns=columns)
df_ri['Review Scores Rating'] = df_ri['Review Scores Rating'].round(4)
df_ri['Average Score of Total'] = df_ri['Average Score of Total'].round(4)
df_ri

,Type of Host,Review Scores Rating,Average Score of Total
0,Single_Host,4.8928,4.8731
1,Small_Host(between 2-10 Listings),4.8441,4.8731
2,Large_Host(over 10 Listings),4.7564,4.8731


In [36]:
total_ratings_df = host_ratings_dfs["review_scores_rating"]
single_host_df = single_listing_host_ratings_dfs["review_scores_rating"]
small_host_df = small_host_ratings_dfs["review_scores_rating"]
large_host_df = large_host_ratings_dfs["review_scores_rating"]

total_df_ratings_c = host_ratings_df_c["review_scores_rating"]
single_host_c = single_listing_host_ratings_dfs_c["review_scores_rating"]
small_host_c = small_host_ratings_c["review_scores_rating"]
large_host_c = large_host_ratings_c["review_scores_rating"]

total_ratings_df_ri = host_ratings_dfs_ri["review_scores_rating"]
large_host_ri = large_host_drop_ri["review_scores_rating"]
small_host_ri = small_host_drop_ri["review_scores_rating"]
single_host_ri = single_host_drop_ri["review_scores_rating"]

data_all = pd.DataFrame({
    'group1': total_ratings_df,
    'group2': total_df_ratings_c,
    'group3': total_ratings_df_ri,
})

all_three_hosts = pd.concat([data_all['group1'], data_all['group2'], data_all['group3']], ignore_index=True, axis=0, names=['All scores all Hosts'])
all_three_hosts = all_three_hosts.dropna()

data_all_single = pd.DataFrame({
    'group1': single_host_df,
    'group2': single_host_c,
    'group3': single_host_ri,})

all_single_hosts = pd.concat([data_all_single['group1'], data_all_single['group2'], data_all_single['group3']], ignore_index=True, axis=0, names=['All scores Single Hosts'])
all_single_hosts = all_single_hosts.dropna()

data_all_small = pd.DataFrame({
    'group1': small_host_df,
    'group2': small_host_c,
    'group3': small_host_ri,})

all_small_hosts = pd.concat([data_all_small['group1'], data_all_small['group2'], data_all_small['group3']], ignore_index=True, axis=0, names=['All scores Small Hosts'])
all_small_hosts = all_small_hosts.dropna()
 
    
data_all_large = pd.DataFrame({
    'group1': large_host_df,
    'group2': large_host_c,
    'group3': large_host_ri,})

all_large_hosts = pd.concat([data_all_large['group1'], data_all_large['group2'], data_all_large['group3']], ignore_index=True, axis=0, names=['All scores Large Hosts'])
all_large_hosts = all_large_hosts.dropna()

In [37]:
# ANOVA Testing

stats.f_oneway(all_three_hosts, all_single_hosts, all_small_hosts, all_large_hosts) 

F_onewayResult(statistic=53.95856177323199, pvalue=1.5555482017296546e-34)

In [38]:
result = stats.kruskal(all_three_hosts, all_single_hosts, all_small_hosts, all_large_hosts)
result

KruskalResult(statistic=172.64088684183346, pvalue=3.4238421303801937e-37)

In [39]:
print("Single Group for All 3 Groups: " + str(stats.ttest_1samp(all_single_hosts, all_three_hosts.mean())))
print("Small Group for All 3 Groups: " + str(stats.ttest_1samp(all_small_hosts, all_three_hosts.mean())))
print("Large Group for All 3 Groups: " + str(stats.ttest_1samp(all_large_hosts, all_three_hosts.mean())))

Single Group for All 3 Groups: TtestResult(statistic=6.28227183897393, pvalue=4.02368886095963e-10, df=2158)
Small Group for All 3 Groups: TtestResult(statistic=-4.819073487655358, pvalue=1.6166802579077205e-06, df=1266)
Large Group for All 3 Groups: TtestResult(statistic=-7.83398882816857, pvalue=1.646131686628289e-13, df=234)


In [40]:
merge = pd.merge(df_a,df_c, on=['Type of Host']).merge(df_ri,on=['Type of Host'])
merge
merge['Mean Review Score Rating'] = merge[['Review Scores Rating_x', 'Review Scores Rating_y',
                       'Review Scores Rating']].mean(axis=1)
merge
merge['Mean Average Total Score'] = merge[['Average Score of Total_x', 'Average Score of Total_y',
                       'Average Score of Total']].mean(axis=1)
merge
columns_to_drop = [
    'Review Scores Rating_x', 'Average Score of Total_x',
    'Review Scores Rating_y', 'Average Score of Total_y',
    'Review Scores Rating', 'Average Score of Total'
]

merge_fin = merge.drop(columns=columns_to_drop)
merge_fin

,Type of Host,Mean Review Score Rating,Mean Average Total Score
0,Single_Host,4.872600,4.854233
1,Small_Host(between 2-10 Listings),4.835600,4.854233
2,Large_Host(over 10 Listings),4.763033,4.854233
